 <center>
    <h1>Bank Loan Default Case</h1>
</center>

**Background:** 
The loan default dataset has 8 variables and 850 records, each record being loan default status for each customer. Each Applicant was rated as “Defaulted” or “Not-Defaulted”. New applicants for loan application can also be evaluated on these 8 predictor variables and classified as a default or non-default based on predictor variables.  

# Importing Libraries

In [ ]:
import os
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from  matplotlib import pyplot 
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
# from sklearn.datasets import make_classification
sns.set()
%matplotlib inline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn import svm
from sklearn.preprocessing import scale
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier


# Working directory

In [ ]:
os.chdir('F:/Rittal Docs/edwisor.com/02.Portfolio/02.Project-2')

# Loading dataset

In [ ]:
df = pd.read_csv("bank-loan.csv")


# Exploratory data analysis

In [ ]:
df.columns

In [ ]:
############ univariate analysis and bivariate analysis ##########################
# analysis for single variable in the dataset and relation between 2 variables.
sns.distplot(df["age"], kde = True , bins = 40)

In [ ]:
sns.distplot(df["income"], kde = True , bins = 40)

In [ ]:
sns.distplot(df["othdebt"], kde = True , bins = 40)

In [ ]:
sns.distplot(df["creddebt"], kde = True , bins = 40)

In [ ]:
sns.distplot(df["debtinc"], kde = True , bins = 40)

In [ ]:
sns.countplot( x = "default" , data = df)

In [ ]:
sns.countplot( x = "ed" , data = df, hue = "default")

In [ ]:
sns.set_style("whitegrid")
plt.figure(figsize = (15,10))
sns.countplot( x = "income" , data = df, hue = "default")

In [ ]:
#checking the relationship between all numeric variable using pair plot
plt.figure(figsize=(14,5))
sns.pairplot(df[:])

## Bivariate Analysis
- Bivariate Analysis - Numeric(TTest)/ Categorical(Chisquare)
- Bivariate Analysis - Visualization
- Variable Reduction - Multicollinearity

In [ ]:
## performing the independent t test on numerical variables

tstats_df = pd.DataFrame()
numeric_var_names=df

for eachvariable in numeric_var_names:
    tstats = stats.ttest_ind(df.loc[df["default"] == 1,eachvariable],df.loc[df["default"] == 0, eachvariable],equal_var=False)
    temp = pd.DataFrame([eachvariable, tstats[0], tstats[1]]).T
    temp.columns = ['Variable Name', 'T-Statistic', 'P-Value']
    tstats_df = pd.concat([tstats_df, temp], axis=0, ignore_index=True)
    
tstats_df =  tstats_df.sort_values(by = "P-Value").reset_index(drop = True)

# P <0.05 therefore we reject null hypo.
Null hypo = 
<br>
Alternate hype = Bothe means from different distributions(which is that the population means are not equal)
<br>
P<0.05 ARE for :
Age=true,Address(P: 0.00000207201)=true,creddebt(P: 0.000000390256)=true,etc

In [ ]:
tstats_df

### Bi-Variate Analysis-Visualization

In [ ]:
def BivariateAnalysisPlot(segment_by):
    """A funtion to analyze the impact of features on the target variable"""
    
    fig, ax = plt.subplots(ncols=1,figsize = (10,8))
    
    #boxplot
    sns.boxplot(x = 'default', y = segment_by, data=df)
    plt.title("Box plot of "+segment_by)
    
    
    plt.show()
    

In [ ]:
BivariateAnalysisPlot("age")
#For default 0 the average age is 35, default 1 is 32

In [ ]:
BivariateAnalysisPlot("ed")
#Both Same

In [ ]:
BivariateAnalysisPlot("employ")
##For default 0 the avrage eploy is 8, default 1 is 3

In [ ]:
BivariateAnalysisPlot("address")
#For default 0 its 8, for default 1 its 5

In [ ]:
BivariateAnalysisPlot("income")
#For default 0 its 35, for default 1 its 25

In [ ]:
BivariateAnalysisPlot("debtinc")
#For default 0 its 8, for default 1 its 13

In [ ]:
BivariateAnalysisPlot("creddebt")
#For default 0 its 0.5, for default 1 its 1.5

In [ ]:
BivariateAnalysisPlot("othdebt")

 ### Variable Reduction - Multicollinearity Check

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
features = "+".join(df.columns.difference(["default"]))

In [ ]:
features

In [ ]:
#perform vif

a, b = dmatrices(formula_like= 'default ~ ' + features,data=df,return_type="dataframe")
vif = pd.DataFrame()

vif["VIF Factor"] = [variance_inflation_factor(b.values, i) for i in range(b.shape[1])]
vif["Features"] = b.columns

In [ ]:
vif

### Observations
----

- There are 850 observations and 9 features in the data set
- All 9 features are numerical in nature
- Out of 850 customers data, 700 are existing customers and 150 are new customers
- In the 700 existing customers, 517 customers are tagged as non defaulters and remaining 183 are tagged as defaulters
- The data is highly imbalanced
- From VIF check, found out that the correlation between the variables is within the acceptable limits

# High Level Analysis

In [ ]:
df.head()

1. Age-Age of each customer                      Numerical
2. Education-Education categories                Categorical
3. Employment-Employment status -                Numerical
    Corresponds to job
    status and being
    converted to numeric
    format
4. Address-Geographic area -                     Numerical
    Converted to numeric
    values
5  Income-Gross Income of each                   Numerical
    customer
6. debtinc-Individual’s debt                     Numerical
    payment to his or her
    gross income
7. creddebt debt-to-credit ratio is a            Numerical
    measurement of how
    much you owe your
    creditors as a
    percentage of your
    available credit (credit
    limits)
8. othdebt-Any other debts                       Numerical




In [ ]:
df.info()

In [ ]:
df.describe()

# Null Check

In [ ]:
# null values present=====> 150 values are missing
# [df["default"].isna()]
df.isnull().sum()

# Missing value analysis

In [ ]:
#Create dataframe with missing percentage
missing_val = pd.DataFrame(df.isnull().sum())

#Reset index
missing_val = missing_val.reset_index()

#Rename variable
missing_val = missing_val.rename(columns = {'index': 'Variables', 0: 'Missing_percentage'})

#Calculate percentage
missing_val['Missing_percentage'] = (missing_val['Missing_percentage']/len(df))*100

#descending order
missing_val = missing_val.sort_values('Missing_percentage', ascending = False).reset_index(drop = True)


#save output results 
missing_val.to_csv("Mising_perc_python.csv", index = False)
missing_val

In [ ]:
# Bar Graph for missing values

sns.barplot( x='Variables', y='Missing_percentage', data = missing_val)

In [ ]:
sns.heatmap(df.isnull(), yticklabels = False, cmap = "viridis")

In [ ]:
df.isnull().sum().sort_values( ascending = False)

In [ ]:
#Impute with median -unsure because cannot fill in missing values for them
# df['default'] = df['default'].fillna(df['default'].median())

# Null correction

In [ ]:
df.default = df.default.fillna(2)
df.info()

In [ ]:
df.default = df.default.astype(np.int64)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

# Outlier Analysis

In [ ]:
#Plot boxplot to visualize Outliers
%matplotlib inline  
l = df.columns.values
number_of_columns=9
number_of_rows = len(l)-1/number_of_columns
plt.figure(figsize=(number_of_columns,5*number_of_rows))
for i in range(0,len(l)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.set_style('whitegrid')
    sns.boxplot(df[l[i]],color='red',orient='v')
    plt.tight_layout()


In [ ]:
#To check distribution-Skewness
plt.figure(figsize=(2*number_of_columns,5*number_of_rows))
for i in range(0,len(l)):
    plt.subplot(number_of_rows + 1,number_of_columns,i+1)
    sns.distplot(df[l[i]],kde=True) 
#All independent variables are right skewed/positively skewed.

In [ ]:
df.shape

In [ ]:
df.head(4)

In [ ]:
%matplotlib inline

plt.boxplot(df['income']) 

# Outlier treatment
#clip_upper() is used to trim values at specified input threshold. We use this function to trim all the values above the threshold of the input value to the specified input value.
#quantile() function return values at the given quantile over requested axis, a numpy.percentile.
#Trim as in it deletes the outlier and replaces it with the upper limit value

In [ ]:
def outlier_capping(x):
    """A funtion to remove and replace the outliers for numerical columns"""
    x = x.clip_upper(x.quantile(0.95))
    
    return(x)

In [ ]:
#outlier treatment
df = df.apply(lambda x: outlier_capping(x))

In [ ]:
df.isnull().any()

In [ ]:
df.info()

# Feature Selection

In [ ]:
# tm = df.groupby("default").mean()
# tm
# df.groupby("default")["employ"].describe()
# df.groupby("default")["age"].describe()
# df.groupby("default")["ed"].describe()
df.groupby("default")["address"].describe()
# df.groupby("default")["income"].describe() 
# df.groupby("default")["debtinc"].describe()
# df.groupby("default")["creddebt"].describe()
# df.groupby("default")["othdebt"].describe()

In [ ]:
df_corr = df
#Set the width and height of the plot
f, ax = plt.subplots(figsize=(12, 7))
#Set the width and height of the plot
f, ax = plt.subplots(figsize=(12, 7))
#Generate correlation matrix
corr = df_corr.corr()
#Plot using seaborn library
sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap='Blues',
            square=True,annot= True, ax=ax)
# Dark shades represents positive correlation while lighter shades represents negative correlation.


In [ ]:
#default correlation matrix
k = 8 #number of variables for heatmap
cols = df.corr().nlargest(k, 'default')['default'].index
cm = df[cols].corr()
plt.figure(figsize=(10,6))
sns.heatmap(cm, annot=True, cmap = 'viridis')

 As we can observe in plot, none of variables are identical to each other, it means that these variables are not highly correlated variables,  so we have to consider all variables for model developement as all variables are important now.

### Chi-Square test of independence-for select categorical variables

Save categorical variables

In [ ]:
df.dtypes

As we can see above, variables__ 'ed' __and__ 'default'__ are only categorical variables, so lets convert them to factor dtype

In [ ]:
#Converting 'ed' and 'default' into categorical type
df['ed'] = pd.Categorical(df['ed'])
print(df.ed.dtype)

df['default'] = pd.Categorical(df['default'])
print(df.default.dtype)

#print(marketing_train[[i]])
#marketing_train.iloc[:,i]=marketing_train.iloc[:,i].cat.codes

In [ ]:
df.dtypes

In [ ]:
cat_names = ["ed"]

In [ ]:
from scipy.stats import chi2_contingency  # for chi-sqr test and comtingency table

In [ ]:
#loop for chi square values   # p = 0.05,  if p < 0.05, Reject Null hyp., Consider that var

from scipy.stats import chi2_contingency  # for chi-sqr test and comtingency table

for i in cat_names:
    print(i)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df['default'], df[i]))
    print(p)

 As we can, see , p Value of cat var (ed) is < 0.05 , It means We reject Null Hypothesis saying that these two variables are important to each other and we cannot drop any one of them.

# Feature Scaling 
 scale the imporant features in measurable units
 1) Scaling by Normalization
or_2) Scaling by Standardization

In most data sets, we will have features with high varying range, units and magnitudes. Features with greater magnitudes will carry a greater value than those with low magnitudes. To nullify this effect, feature scaling is performed.

In [ ]:
#check Normality by Histogram Before Normalization / Standerdization

%matplotlib inline  
plt.hist(df['age'], bins='auto')

In [ ]:
#Verifying  it again
    
%matplotlib inline  
plt.hist(df['income'], bins='auto')

 Since we can see that , data is not normallaly distributed , Hence go for **Normalization** 1st instead of Standardisation

In [ ]:
cnames = ['age', 'employ', 'address', 'income', 'debtinc', 'creddebt', 'othdebt']


In [ ]:
cnames

In [ ]:
#Nomalisation

for i in cnames:
    print(i)
    df[i] = (df[i] - min(df[i]))/(max(df[i]) - min(df[i]))

In [ ]:
df.head(2)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
features = df.columns
importances = best_rf_clf.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

In [ ]:
#### selecting all missing values from dataset and we will predict those default case with best accurate 

train = df.loc[df['default'] != 2]
print(train.head())
print(train.tail())

In [ ]:
train.default.unique()

In [ ]:
test = df.loc[df.default == 2]
test = test.iloc[:, 0:8]
print(test.head(2))
print(test.tail(2))

In [ ]:
df.info()

# Separate X and Y

In [ ]:
X = train.drop(['default'], axis=1)
y = train['default']

# Modeling 

In [ ]:
#dividing data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test,y_train, y_test = train_test_split(X,y, test_size=0.25, random_state=42)

In [ ]:
##################################### Logistic Regression ####################################################

from sklearn.linear_model import LogisticRegression
logreg =LogisticRegression(penalty='l2', C=0.1)
logreg.fit(X_train,y_train)
#predict new test cases
Log_Predictions = logreg.predict(X_test)
print(logreg.score(X_test,y_test))

In [ ]:
C_space = np.array([0.0001, 0.001, 0.1, 1])

In [ ]:
param_grid = {'C' : C_space}

In [ ]:
clf_logis_tuning = GridSearchCV(logreg, param_grid, cv=5)

In [ ]:
clf_logis_tuning.fit(X_train, y_train)

In [ ]:
print("Tuned Logistic Regression Parameters: {}".format(clf_logis_tuning.best_params_)) 
print("Best score is {}".format(clf_logis_tuning.best_score_))

Using Logistic Regression, we achieved a training set accuracy of ~79% (in terms of training set accuracy alone.)

Next, the model will be fit with the best parameter of C and then it will be used to predict the test set.

In [ ]:
Log_Predictions = logreg.predict(X_test)

In [ ]:
p_logreg_ba = logreg.predict_proba(X_test)

In [ ]:
accuracy_score(y_test, Log_Predictions)

In [ ]:
print(classification_report(y_test, Log_Predictions))

In [ ]:
precision_lg, recall_lg, thresholds_lg = precision_recall_curve(y_test, p_logreg_ba[:, 1])

In [ ]:
fpr_lg, tpr_lg, thresholds_lg = roc_curve(y_test, p_logreg_ba[:, 1])

In [ ]:
#build confusion matrix

CM = pd.crosstab(y_test,Log_Predictions)

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(ytest, y_preds)*100
print(((TP+TN)*100)/(TP+TN+FP+FN))

#False Negative rate 
print((FN*100)/(FN+TP))

print("Defaulted", sum(Log_Predictions!=0))
print("Non-defaulted ", sum(Log_Predictions==0))
CM

In [ ]:
#ROC curve and AUC
prob=logreg.predict_proba(X_test)
# print(prob)
prob=prob[:,0]
auc=roc_auc_score(y_test,prob)
print(auc)
fpr,tpr,thresh=roc_curve(y_test,prob)
plt.plot([0,1],[1,0],linestyle='--')
plt.plot(fpr,tpr,marker='.',color='red',label="Logistic regression")

In [ ]:
###################################### Decision Tree #####################################
from sklearn import tree
C50_model = tree.DecisionTreeClassifier(criterion='entropy').fit(X_train, y_train)

#predict new test cases
C50_Predictions = C50_model.predict(X_test)

In [ ]:
#  plotting decision tree
from sklearn.externals.six import StringIO  
from IPython.display import Image  
from sklearn.tree import export_graphviz
import pydotplus
dot_data = StringIO()
export_graphviz(C50_model, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
Image(graph.create_png())

In [ ]:
#build confusion matrix
# from sklearn.metrics import confusion_matrix 
# CM = confusion_matrix(y_test, y_pred)
CM = pd.crosstab(y_test, C50_Predictions)

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(y_test, y_pred)*100
print(((TP+TN)*100)/(TP+TN+FP+FN))

#False Negative rate 
print((FN*100)/(FN+TP))

print("Defaulted", sum(C50_Predictions!=0))
print("Non-defaulted ", sum(C50_Predictions==0))
#Results
CM

In [ ]:
print(classification_report(y_test, C50_Predictions))

In [ ]:
dt_predict1 =C50_model.predict_proba(X_test)[:, 1]

In [ ]:
precision_dc, recall_dc, thresholds_dc = precision_recall_curve(y_test,dt_predict1)

In [ ]:
fpr_dc, tpr_dc, thresholds_dc = roc_curve(y_test,C50_Predictions)

In [ ]:
#ROC curve and AUC
prob=C50_model.predict_proba(X_test)
# print(prob)
prob=prob[:,0]
auc=roc_auc_score(y_test,prob)
print(auc)
fpr,tpr,thresh=roc_curve(y_test,prob)
plt.plot([0,1],[1,0],linestyle='--')
plt.plot(fpr,tpr,marker='.',color='red')

In [ ]:
###################################### Random Forest #####################################
from sklearn.ensemble import RandomForestClassifier

RF_model = RandomForestClassifier(n_estimators = 200).fit(X_train, y_train)

In [ ]:
RF_Predictions = RF_model.predict(X_test)

In [ ]:
cv_score = cross_val_score(RF_model, X_train, y_train, cv = 5)
print("Average 5-fold CV Score: {}".format(np.mean(cv_score)))

In [ ]:
cv_score = cross_val_score(RF_model, X_train, y_train, cv = 5, scoring = 'roc_auc')
print("Average 5-fold CV Score using ROC_AUC: {}".format(np.mean(cv_score)))

In [ ]:
accuracy_score(y_test, RF_Predictions)

In [ ]:
n_space = np.array([5, 6, 10, 12, 15, 50, 100, 200, 500])
criterion_vals = ['gini', 'entropy']
max_features_vals = ['auto', 'sqrt', 'log2']
min_samples_leaf_sp = [1, 5, 10, 25, 50]
bootstrap_sp = [True, False]


param_grid = {'n_estimators': n_space, 'criterion' : criterion_vals, 
              'max_features':max_features_vals, 'min_samples_leaf': min_samples_leaf_sp, 
              'bootstrap': bootstrap_sp}

In [ ]:
rf_clf_tuning = GridSearchCV(RF_model, param_grid, cv = 5)

In [ ]:
rf_clf_tuning.fit(X_train, y_train)

In [ ]:
print("Tuned RF Parameters: {}".format(rf_clf_tuning.best_params_)) 
print("Best score is {}".format(rf_clf_tuning.best_score_))

In [ ]:
best_rf_clf = RandomForestClassifier(criterion = 'gini', bootstrap = True, 
                                     max_features = 'log2', min_samples_leaf = 5, n_estimators = 200)

In [ ]:
best_rf_clf.fit(X_train, y_train)

In [ ]:
y_best_rf_preds = best_rf_clf.predict(X_test)

In [ ]:
cv_score = cross_val_score(best_rf_clf, X_train, y_train, cv = 5)
print("Average 5-fold CV Score: {}".format(np.mean(cv_score)))

In [ ]:
accuracy_score(y_test, y_best_rf_preds)

In [ ]:
y_best_proba = best_rf_clf.predict_proba(X_test)[:, 1]

In [ ]:
fpr_rf, tpr_rf, thresholds_rf = roc_curve(y_test, y_best_proba)

In [ ]:
print(classification_report(y_test, (y_best_proba > 0.5).astype(int)))

In [ ]:
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(y_test, y_best_proba)

In [ ]:
#build confusion matrix
# from sklearn.metrics import confusion_matrix 
# CM = confusion_matrix(y_test, y_pred)
CM = pd.crosstab(y_test, y_best_rf_preds)

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(y_test, y_pred)*100
print(((TP+TN)*100)/(TP+TN+FP+FN))

#False Negative rate 
print((FN*100)/(FN+TP))

print("Defaulted", sum(y_best_rf_preds!=0))
print("Non-defaulted ", sum(y_best_rf_preds==0))
CM

In [ ]:
#ROC curve and AUC
prob=best_rf_clf.predict_proba(X_test)
# print(prob)
prob=prob[:,0]
auc=roc_auc_score(y_test,prob)
print(auc)
fpr,tpr,thresh=roc_curve(y_test,prob)
plt.plot([0,1],[1,0],linestyle='--')
plt.plot(fpr,tpr,marker='.',color='red')

In [ ]:
###################################### Gradient Boosting Algorithm #####################################
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf_xgb = GradientBoostingClassifier(n_estimators=200)

In [ ]:
clf_xgb.fit(X_train, y_train)

In [ ]:
y_predict_xgb = clf_xgb.predict(X_test)

In [ ]:
accuracy_score(y_test, y_predict_xgb)

In [ ]:
param_grid = {'n_estimators':range(20,81,10)}

In [ ]:
clf_xgb_tuning = GridSearchCV(clf_xgb, param_grid, cv=5)

In [ ]:
clf_xgb_tuning.fit(X_train, y_train)

In [ ]:
print("Tuned Gradient Boosting Parameters: {}".format(clf_xgb_tuning.best_params_)) 
print("Best score is {}".format(clf_xgb_tuning.best_score_))

In [ ]:
clf_xgb_best = GradientBoostingClassifier(n_estimators=80)

In [ ]:
clf_xgb_best.fit(X_train, y_train)

In [ ]:
y_predict_xgb_best = clf_xgb_best.predict(X_test)

In [ ]:
accuracy_score(y_test, y_predict_xgb_best)

In [ ]:
y_predict_xgb1 = clf_xgb_best.predict_proba(X_test)[:, 1]

In [ ]:
print(classification_report(y_test, y_predict_xgb_best))

In [ ]:
precision_xgb, recall_xgb, thresholds_xgb = precision_recall_curve(y_test, y_predict_xgb1)

In [ ]:
fpr_xgb, tpr_xgb, thresholds_xgb = roc_curve(y_test, y_predict_xgb1)

In [ ]:
#build confusion matrix

CM = pd.crosstab(y_test, y_predict_xgb_best)

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(ytest, dt_pred)*100
print(((TP+TN)*100)/(TP+TN+FP+FN))

#False Negative rate 
print((FN*100)/(FN+TP))

print("Defaulted", sum(y_predict_xgb_best!=0))
print("Non-defaulted ", sum(y_predict_xgb_best==0))
CM

In [ ]:
#ROC curve and AUC
prob=clf_xgb.predict_proba(X_test)
# print(prob)
prob=prob[:,0]
auc=roc_auc_score(y_test,prob)
print(auc)
fpr,tpr,thresh=roc_curve(y_test,prob)
plt.plot([0,1],[1,0],linestyle='--')
plt.plot(fpr,tpr,marker='.',color='red')

In [ ]:
###################################### XGB classifier #####################################
from xgboost import XGBClassifier
XG_model = XGBClassifier(n_estimators = 200).fit(X_train, y_train)

In [ ]:
XG_Predictions = XG_model.predict(X_test)

In [ ]:
#build confusion matrix
# from sklearn.metrics import confusion_matrix 
# CM = confusion_matrix(y_test, y_pred)
CM = pd.crosstab(y_test, XG_Predictions)

#let us save TP, TN, FP, FN
TN = CM.iloc[0,0]
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FP = CM.iloc[0,1]

#check accuracy of model
#accuracy_score(y_test, y_pred)*100
print(((TP+TN)*100)/(TP+TN+FP+FN))

#False Negative rate 
print((FN*100)/(FN+TP))

print("Defaulted", sum(XG_Predictions!=0))
print("Non-defaulted ", sum(XG_Predictions==0))
CM

In [ ]:
print(classification_report(y_test, XG_Predictions))

In [ ]:
XG_predict1 =XG_model.predict_proba(X_test)[:, 1]

In [ ]:
precision_XG, recall_XG, thresholds_XG = precision_recall_curve(y_test,XG_predict1)

In [ ]:
fpr_XG, tpr_XG, thresholds_XG = roc_curve(y_test,XG_predict1)

In [ ]:
#ROC curve and AUC
prob=XG_model.predict_proba(X_test)
# print(prob)
prob=prob[:,0]
auc=roc_auc_score(y_test,prob)
print(auc)
fpr,tpr,thresh=roc_curve(y_test,prob)
plt.plot([0,1],[1,0],linestyle='--')
plt.plot(fpr,tpr,marker='.',color='red')

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.plot(recall_lg, precision_lg)
plt.plot(recall_rf, precision_rf)
plt.plot(recall_dc, precision_dc)
plt.plot(recall_xgb, precision_xgb)
plt.plot(recall_XG,precision_XG)

plt.legend(('Logistic Reg', 'RandomForestClassifier', 'Decision Tree','Gradient Boosting','XG Boosting'))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision vs. Recall curve')

In [ ]:
area_log_reg = auc(recall_lg, precision_lg)
print(area_log_reg)
area_dc = auc(recall_dc, precision_dc)
print(area_dc)
area_rf = auc(recall_rf, precision_rf)
print(area_rf)
area_xgb = auc(recall_xgb, precision_xgb)
print(area_xgb)
area_XG = auc(recall_XG,precision_XG)
print(area_xgb)


In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
plt.plot(fpr_lg, tpr_lg)
plt.plot(fpr_dc, tpr_dc)
plt.plot(fpr_rf, tpr_rf)
plt.plot(fpr_xgb, tpr_xgb)
plt.plot(fpr_XG, tpr_XG)
plt.legend(('Logistic Regression' , 'Decision tree', 'RandomForestClassifier', 'Gradient Boosting','XG Boosting'))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')

In [ ]:
Areas_ROC_logistic  = roc_auc_score(y_test, Log_Predictions)
Areas_ROC_decision = roc_auc_score(y_test,C50_Predictions)
Areas_ROC_randomforest = roc_auc_score(y_test, y_best_rf_preds)
Areas_ROC_xgb = roc_auc_score(y_test, y_predict_xgb_best)
Areas_ROC_XG = roc_auc_score(y_test,XG_predict1)
print(Areas_ROC_logistic)
print(Areas_ROC_decision)
print(Areas_ROC_randomforest)
print(Areas_ROC_xgb)
print(Areas_ROC_XG)

In [ ]:
#Saving Prediction Values to local file.

result=pd.DataFrame(y_test)
result['pred_def'] = (Log_Predictions)

result.to_csv("LG output python.csv",index=False)

## Data Visualizations

In [ ]:
# -Jointplots for Bivariate Analysis.
# -Here Scatter plot has regression line between 2 variables along with separate Bar plots of both variables.
plt.figure(figsize=(20,10))
_ = sns.jointplot(x='default',y='income', data=df, kind = 'hex')

In [ ]:
# Relationship between number of age and income

plt.figure(figsize=(5,5))
plt.scatter(x=df['age'], y=df['income'], s=10)
plt.xlabel('Age')
plt.ylabel('income')
plt.show()

In [ ]:
# Relationship beetween number of default and othdebt

plt.figure(figsize=(5,5))
plt.scatter(x=df['default'], y=df['othdebt'], s=10)
plt.xlabel('Age')
plt.ylabel('income')
plt.show()

In [ ]:
# Relationship beetween number of employment and income

plt.figure(figsize=(5,5))
plt.scatter(x=df['employ'], y=df['income'], s=10)
plt.xlabel('employed')
plt.ylabel('income')
plt.show()